## Simple example -- pairwise distance

Suppose we have `N` vectors in $\mathbb{R}^3$, stored as a $3 \times N$ matrix and labeled $x_1 \in \mathbb{R}^3$, $x_2 \in \mathbb{R}^3$, ..., $x_N \in \mathbb{R}^3$

Suppose we also have a metric function $\rho: \mathbb{R}^3 \times \mathbb{R}^3 \rightarrow \mathbb{R}^+$  that we would like to apply to each pair of vectors to compute an $N \times N$ matrix $D$, where the $i,j$ element is give by $D_{i,j} = \rho(x_i, x_j)$

Let's start with the Euclidean (L2) norm $\rho(x, y) = \sqrt{\sum_{i=1}^3(x_i - y_i)^2}$

Here's how we might compute this in Julia

In [16]:
N = 1500
X = randn(3, N)  # create the matrix of random numbers drawn from N(0, 1)

3×1500 Array{Float64,2}:
 -0.486749  0.0245151   1.34225   …  -0.536434   0.426064  -0.641357
  1.92775   0.344174   -0.949785      2.03927    0.744031  -0.967209
 -1.83757   1.39897     0.738193      1.11685   -0.844427   0.022634

In [19]:
function computeD(ρ, X)
    m, N = size(X)
    out = zeros(N, N)
    for j in 1:N, i in 1:N
        # [index...] notation. Colon means "everything" in dimension
        out[i, j] = ρ(X[:, i], X[:, j])
    end
    out # implicit return of last expression in function
end

# short-hand function
# the `.+` and `.^` apply "elementwise" to arguments
# Can work with other functions `f` like `f.(arg1, arg2, arg3, ...)`
ρ_l2(x, y) = sqrt(sum((x.-y).^2))

@time D = computeD(ρ_l2, X)

  0.814886 seconds (9.05 M allocations: 809.070 MiB, 5.82% gc time)


1500×1500 Array{Float64,2}:
 0.0      3.63927  4.27317  5.36205  3.25066   …  2.95694  1.79464   3.44456 
 3.63927  0.0      1.96147  3.07954  1.3727       1.80766  2.31386   2.0143  
 4.27317  1.96147  0.0      2.08418  1.8886       3.55068  2.49261   2.1088  
 5.36205  3.07954  2.08418  0.0      2.3574       4.7799   3.8158    2.06075 
 3.25066  1.3727   1.8886   2.3574   0.0          2.59603  1.91758   0.660686
 2.59047  1.5745   1.82719  2.90138  0.907678  …  2.43719  1.03419   1.27267 
 4.62637  2.30024  2.38274  1.45163  1.47734      3.77905  3.33918   1.24347 
 2.55225  2.01782  2.88717  3.25543  1.08482      2.43454  1.84727   1.21248 
 3.64518  1.1125   2.48884  2.83331  0.990152     2.18667  2.57456   1.4553  
 3.65218  3.49078  3.81227  3.20425  2.22233      4.13203  3.19039   1.73503 
 3.00428  2.64668  1.77791  3.40994  2.33841   …  3.44643  1.34646   2.52786 
 3.6116   4.03071  2.78767  3.84434  3.37138      4.83928  2.40468   3.28314 
 4.03737  3.81977  3.72171  2.87819 

Let's test our work  by showing the diagonal is all zeros:


In [20]:
using LinearAlgebra: diag

# notice  the `.` notation for `abs`
maximum(abs.(diag(D) .- zeros(N)))

0.0

## Python implementation

So that we can gauge performance

In [21]:
using PyCall

In [22]:
py"""
import numpy as np

def ρ_l2(x, y):
    return np.sqrt(np.sum((x - y) ** 2))


def computeD(X, ρ=ρ_l2):
    m, N = X.shape
    out = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            out[i, j] = ρ(X[:, i], X[:, j])
    return out
"""

py_computeD = py"computeD"

PyObject <function computeD at 0x7f6026439ae8>

In [23]:
@time py_D = py_computeD(X)

 18.615087 seconds (45 allocations: 17.168 MiB, 0.08% gc time)


1500×1500 Array{Float64,2}:
 0.0      3.63927  4.27317  5.36205  3.25066   …  2.95694  1.79464   3.44456 
 3.63927  0.0      1.96147  3.07954  1.3727       1.80766  2.31386   2.0143  
 4.27317  1.96147  0.0      2.08418  1.8886       3.55068  2.49261   2.1088  
 5.36205  3.07954  2.08418  0.0      2.3574       4.7799   3.8158    2.06075 
 3.25066  1.3727   1.8886   2.3574   0.0          2.59603  1.91758   0.660686
 2.59047  1.5745   1.82719  2.90138  0.907678  …  2.43719  1.03419   1.27267 
 4.62637  2.30024  2.38274  1.45163  1.47734      3.77905  3.33918   1.24347 
 2.55225  2.01782  2.88717  3.25543  1.08482      2.43454  1.84727   1.21248 
 3.64518  1.1125   2.48884  2.83331  0.990152     2.18667  2.57456   1.4553  
 3.65218  3.49078  3.81227  3.20425  2.22233      4.13203  3.19039   1.73503 
 3.00428  2.64668  1.77791  3.40994  2.33841   …  3.44643  1.34646   2.52786 
 3.6116   4.03071  2.78767  3.84434  3.37138      4.83928  2.40468   3.28314 
 4.03737  3.81977  3.72171  2.87819 

In [24]:
maximum(abs.(D .- py_D))

0.0

On one run on my machine the Julia version took 1.49 seconds whereas the Python one took 18.76 seconds. 

This was a pretty much "free" speedup as the code was almost the same for both languages. It is not uncommon to se factors of 100 or 1000 when moving hand-written Python code to Julia

Note that the code in each example was not meant to reach optimal performance, both implementations could be improved

## Faster Julia version



In [25]:
function computeD_l2(X)
    m, N = size(X)
    out = zeros(N, N)
    @inbounds for j in 1:N, i in 1:N
        # [index...] notation. Colon means "everything" in dimension
        val = 0.0
        @simd for d in 1:m
            val += (X[d, i] - X[d, j])^2
        end
        out[i, j] = sqrt(val)
    end
    out # implicit return of last expression in function
end

computeD_l2 (generic function with 1 method)

In [33]:
@time D2 = computeD_l2(X);

  0.017628 seconds (6 allocations: 17.166 MiB, 28.30% gc time)


In [34]:
maximum(abs.(D .- D2))

8.881784197001252e-16

On my machine this version of code ran the computation in 0.0095 seconds -- that's a 157x speedup over the original Julia and a 1978x speedup over the Python

With Julia it is possible to write "lower level" code and get drastically better performance

Sometimes you might hear that "in Julia you can get C-like performance if you write C-like code"

This is typically not possible in other common data analytics languages like Python, Matlab, or R